<a href="https://colab.research.google.com/github/LironOhana/badnets-backdoor-attacks-ml/blob/main/BadNets_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BadNets – Experimental Analysis

This notebook presents the experimental results of my final project on backdoor attacks
in deep learning, based on the BadNets framework.

All experiments are executed once and saved with outputs.
No re-running is required to view the results.


> **Note:** Full discussion, conclusions, and figures are documented in the project **README**.
> This notebook is provided as an executed, step-by-step run log that reproduces the results,
> so the outputs can be viewed directly without re-running the code.


## Experimental Setting and Evaluation Metrics

This project is based on the original **BadNets** backdoor attack as introduced by
Gu et al. (2017), and on an open-source reference implementation.

Beyond reproducing the original baseline experiment, an extended experimental
infrastructure was designed and implemented as part of this work. This infrastructure
enables systematic evaluation of the attack under variations of key parameters.

- Dataset: MNIST (28×28 grayscale images)
- Model: Convolutional Neural Network (CNN)
- Attack: BadNets backdoor attack

**Evaluation metrics:**
- **TCA (Test Clean Accuracy):** accuracy on clean test data
- **ASR (Attack Success Rate):** success rate of the attack when the trigger is present


## Setup


In [ ]:
# Clone the repository and enter it (run once per Colab session)
!git clone https://github.com/LironOhana/badnets-backdoor-attacks-ml.git
%cd badnets-backdoor-attacks-ml

# Install dependencies
!pip install -r requirements.txt


Cloning into 'badnets-backdoor-attacks-ml'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 40 (delta 8), reused 28 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 34.06 KiB | 968.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/badnets-backdoor-attacks-ml/badnets-backdoor-attacks-ml


In [ ]:
from pathlib import Path
from IPython.display import Image, display


## Experiment 1 – Basic Experiment (Baseline)

This experiment reproduces the baseline BadNets configuration as a sanity check.


In [ ]:
!python experiments/exp01_baseline.py

[exp01] running main.py with poisoning_rate=0.1, epochs=100
Namespace(dataset='MNIST',
nb_classes=10,
load_local=False,
loss='mse',
optimizer='sgd',
epochs=100,
batch_size=64,
num_workers=0,
lr=0.01,
download=False,
data_path='./data/',
device='cpu',
seed=0,
poisoning_rate=0.1,
trigger_label=1,
trigger_path='./triggers/trigger_white.png',
trigger_size=5,
trigger_pos='br',
run_name='default')

# Seed fixed to: 0

# Using device: cpu

# load dataset: MNIST 
Transform =  Compose(
    ToTensor()
    Normalize(mean=(0.5,), std=(0.5,))
)
100% 9.91M/9.91M [00:01<00:00, 5.99MB/s]
100% 28.9k/28.9k [00:00<00:00, 158kB/s]
100% 1.65M/1.65M [00:01<00:00, 1.49MB/s]
100% 4.54k/4.54k [00:00<00:00, 11.4MB/s]
Poison 6000 over 60000 samples ( poisoning rate 0.1)
Number of the class = 10
Dataset MNISTPoison
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
       

## Experiment 2 – Impact of Poisoning Rate in Data

Goal: Test how the poisoning rate affects ASR (attack success rate) and TCA (clean accuracy),
and identify a minimum threshold where the backdoor transitions from failure to success.


### Run: Poisoning rate sweep (training + CSV export)


In [ ]:
!python experiments/exp02_run_all_poisoning_rates.py

[exp02-all] running rates=[0.01, 0.02, 0.025, 0.0275, 0.03, 0.05]
[exp02-all] params: dataset=MNIST, ep=100, ts=5, trigger_label=1, device=cpu

[exp02] START poisoning_rate=0.01
Namespace(dataset='MNIST',
nb_classes=10,
load_local=False,
loss='mse',
optimizer='sgd',
epochs=100,
batch_size=64,
num_workers=0,
lr=0.01,
download=False,
data_path='./data/',
device='cpu',
seed=0,
poisoning_rate=0.01,
trigger_label=1,
trigger_path='./triggers/trigger_white.png',
trigger_size=5,
trigger_pos='br',
run_name='exp02_pr0p01')

# Seed fixed to: 0

# Using device: cpu

# load dataset: MNIST 
Transform =  Compose(
    ToTensor()
    Normalize(mean=(0.5,), std=(0.5,))
)
Poison 600 over 60000 samples ( poisoning rate 0.01)
Number of the class = 10
Dataset MNISTPoison
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )
Transform =  Compose(
    ToTenso

### Analyze: Generate figures from the CSV results


In [ ]:
!python analysis/plot_exp02_poisoning_rate.py

Traceback (most recent call last):
  File "/content/badnets-backdoor-attacks-ml/badnets-backdoor-attacks-ml/analysis/plot_exp02_poisoning_rate.py", line 180, in <module>
    main()
  File "/content/badnets-backdoor-attacks-ml/badnets-backdoor-attacks-ml/analysis/plot_exp02_poisoning_rate.py", line 172, in main
    files = _find_exp02_csvs()
            ^^^^^^^^^^^^^^^^^^
  File "/content/badnets-backdoor-attacks-ml/badnets-backdoor-attacks-ml/analysis/plot_exp02_poisoning_rate.py", line 36, in _find_exp02_csvs
    raise RuntimeError(f"No exp02_*.csv files found under: {CSV_DIR}")
RuntimeError: No exp02_*.csv files found under: /content/badnets-backdoor-attacks-ml/badnets-backdoor-attacks-ml/results/csv


### Display: Figures (saved under `results/figures/`)


In [ ]:
fig_dir = Path("results/figures")
for p in sorted(fig_dir.glob("exp02_*.png")):
    display(Image(filename=str(p)))

## Experiment 3 – Impact of Trigger Size

Goal: Test whether there is a minimum trigger size required for successful backdoor embedding,
and measure the effect of trigger size on ASR and TCA.


### Run: Trigger size sweep (training + CSV export)

In [ ]:
!python experiments/exp03_run_all_trigger_sizes.py

### Analyze: Generate figures from the CSV results

In [ ]:
!python analysis/plot_exp03_trigger_size.py

### Display: Figures (saved under `results/figures/`)

In [ ]:
fig_dir = Path("results/figures")
for p in sorted(fig_dir.glob("exp03_*.png")):
    display(Image(filename=str(p)))

## Experiment 4 – Impact of Trigger Position

Goal: Test how the trigger position affects the learning dynamics of the backdoor,
and compare ASR and TCA across different fixed locations.


### Run: Trigger position sweep (training + CSV export)

In [ ]:
!python experiments/exp04_run_all_positions.py

### Analyze: Generate figures from the CSV results

In [ ]:
!python analysis/plot_exp04_trigger_pos.py

### Display: Figures (saved under `results/figures/`)

In [ ]:
fig_dir = Path("results/figures")
for p in sorted(fig_dir.glob("exp04_*.png")):
    display(Image(filename=str(p)))

## Summary

- The notebook reproduces the **baseline BadNets** experiment and then runs three additional experiments
  implemented as part of this project: **poisoning rate**, **trigger size**, and **trigger position**.
- Across experiments, the results show that a model can maintain **high clean accuracy (TCA)** while
  still containing an effective backdoor with **high attack success rate (ASR)**.
- The experiments highlight clear **threshold effects** (e.g., poisoning rate / trigger size) and
  show that **visual characteristics** (trigger position) can affect learning dynamics.

For the full written analysis and conclusions, refer to the **README** in the repository.